# GAIA DR2 cross-match

In [1]:
# Authors: Lauren Anderson, Megan Bedell, Chiara Mingarelli
# Please cite Mingarelli, Anderson, Bedell and Spergel (in prep)

In [2]:
# %load query.py
import numpy as np
from astropy import units as u
from astropy.io import ascii
from astroquery.xmatch import XMatch
import requests

In [3]:
# %load query.py
import numpy as np
import glob

from astropy import units as u
from astropy.io import ascii
from astroquery.xmatch import XMatch
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord, ICRS, Galactic, Angle
from astropy.time import Time

## Upload your files with the info you'd like to cross-match with GAIA DR2

In [4]:
files = glob.glob('../IPTA_DR1/*/*.par') # We are interested in pulsars, these files contain their parameters

## Here we parse the pulsar .par files to get the known pulsar positions and proper motions. This is specific to the pulsar data format used by the International Pulsar Timing Array.

In [6]:
RAs = []
DECs = []
Names = []
pmra = []
pmdec = []
pepoch = []
posepoch = []
dmepoch = []
dm = []
coords = []

for file in files:
    d = {}
    with open(file) as f:
        for line in f:
            key = line.split()[0]
            value = line.split()[1]
            d[key] = value
            
    try:
        ra = Angle(d['RAJ'], u.hourangle).to(u.deg)
        dec = Angle(d['DECJ'], u.degree)
        coord = SkyCoord(ra=ra, dec=dec, frame='icrs',
                     pm_ra_cosdec = float(d['PMRA'])*u.mas/u.yr, # bug updated, thanks Paul Ray
                     pm_dec=float(d['PMDEC'])*u.mas/u.yr,
                     obstime=Time(float(d['POSEPOCH']), format='mjd'),
                     distance=1. * u.kpc) # HACK
        coords.append(coord)
        RAs.append(ra.value)
        #print(d['PSRJ'], d['DECJ'])
        DECs.append(dec.value)
        Names.append(d['PSRJ'])
        pmra.append(coord.pm_ra_cosdec.value)
        pmdec.append(coord.pm_dec.value)
        pepoch.append(d['PEPOCH'])
        posepoch.append(d['POSEPOCH'])
        dmepoch.append(d['DMEPOCH'])
        dm.append(d['DM'])
        
    except KeyError:
        lon = float(d['ELONG'])*u.deg
        lat = float(d['ELAT'])*u.deg
        coord = SkyCoord(l=lon, b=lat, frame='galactic',
                         pm_l_cosb = float(d['PMELONG'])*np.cos(lat.to(u.rad))*u.mas/u.yr, 
                         pm_b=float(d['PMELAT'])*u.mas/u.yr,
                         obstime=Time(float(d['POSEPOCH']), format='mjd'),
                         distance=1. * u.kpc) # HACKITY HACK HACK
        icrscoord = coord.transform_to(ICRS)
        coords.append(icrscoord)
        #print(icrscoord)
        #print(d['PSRJ'])
        RAs.append(icrscoord.ra.to(u.deg).value)
        DECs.append(icrscoord.dec.to(u.deg).value)
        Names.append(d['PSRJ'])
        pmra.append(icrscoord.pm_ra_cosdec.value)
        pmdec.append(icrscoord.pm_dec.value)
        pepoch.append(d['PEPOCH'])
        posepoch.append(d['POSEPOCH'])
        dmepoch.append(d['DMEPOCH'])
        dm.append(d['DM'])        
        #coord = SkyCoord(d[''])
        
        continue
        
t = Table([Names, RAs, DECs, pmra, pmdec, pepoch, posepoch, dmepoch, dm], names=['names', 'ra', 'dec', 'pmra', 'pmdec', 'dm', 'pepoch', 'posepoch', 'dmepoch'])        
#tmin = Table([RAs, DECs], names=['ra', 'dec'])


In [7]:
# Test

In [8]:
print(coords[0].apply_space_motion(new_obstime=Time(57023., format='mjd')))

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (7.61427606, 4.86102859, 999.99999999)
 (pm_ra, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (-5.86760677, -0.41012739, 4.3724621e-06)>


In [9]:
gaia_time = Time(57023., format='mjd')

## Here we propagate the pulsar proper motion to update them to the GAIA DR2 reference epoch.

In [10]:
t['ra'] = [c.apply_space_motion(new_obstime=gaia_time).ra.value for c in coords]
t['dec'] = [c.apply_space_motion(new_obstime=gaia_time).dec.value for c in coords]

In [11]:
t.write('ipta_data.csv', format='csv', overwrite=True)


# We tell the x-match server that we want to search GAIA DR2 for objcts at the RA and DEC we specifiy in our file, within a 3 arsecond search radius.

In [21]:
r = requests.post(
'http://cdsxmatch.u-strasbg.fr/xmatch/api/v1/sync',
data={'request': 'xmatch', 'distMaxArcsec': 3.0, 'RESPONSEFORMAT': 'votable',
'cat2': 'vizier:I/345/gaia2', 'colRA1': 'ra', 'colDec1': 'dec'},
files={'cat1': open('ipta_data.csv', 'r')})

In [22]:
# Test
print(r.text)

<?xml version="1.0" encoding="UTF-8"?>
<VOTABLE version="1.2" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
xsi:schemaLocation="http://www.ivoa.net/xml/VOTable/v1.2 http://www.ivoa.net/xml/VOTable/v1.2"
xmlns="http://www.ivoa.net/xml/VOTable/v1.2">
	<DESCRIPTION>
		Generated by the CDS XMatch engine.
		Please report any problem to:
		 - francois-xavier.pineau@astro.unistra.fr or
		 - thomas.boch@astro.unistra.fr or
		 - cds-question@unistra.fr
	</DESCRIPTION>
	<INFO name="queryParameters" value="7">
		request=xmatch
		distmaxarcsec=3.0
		responseformat=votable
		cat2=vizier:I/345/gaia2
		colra1=ra
		coldec1=dec
		cat1=ipta_data.csv
	</INFO>
	<INFO name="nTuples" value="18"/>
	<INFO name="computationTime" value="00:00:00.038"/>
	<RESOURCE type="results">
		<INFO name="QUERY_STATUS" value="OK"/>
		<TABLE ID="xmatchResult" name="xmatchResult" nrows="18">
			<FIELD name="angDist" datatype="double" unit="arcsec" ucd="pos.angDistance">
				<DESCRIPTION>Xmatch distance in arcseconds</

In [23]:
h = open('ipta_crossmatch_3arcsecond.vot', 'w')
h.write(r.text)
h.close()

In [16]:
r.text

'angDist,names,ra,dec,pmra,pmdec,dm,pepoch,posepoch,dmepoch,ra_epoch2000,dec_epoch2000,errHalfMaj,errHalfMin,errPosAng,source_id,ra,ra_error,dec,dec_error,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,duplicated_source,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_mag,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_mag,bp_rp,radial_velocity,radial_velocity_error,teff_val,a_g_val,e_bp_min_rp_val,radius_val,lum_val\n2.129243,J0437-4715,69.31657735576458,-47.25266789914286,121.43993281718666,-71.47620158262019,54500,54500,54500,2.644571992220118,69.31582341820,-47.25237136340,0.026,0.019,0,4789864076732331648,69.31660276366,0.5823,-47.25267777510,0.7385,8.3250,0.6777,122.864,1.197,-71.166,1.670,0,1.29299e+02,1.07676e+00,20.409374,5.45906e+01,6.52378e+00,21.008593,1.32014e+02,8.90693e+00,19.460367,1.548225,,,,,,,\n1.901612,J0613-0200,93.43323479383737,-2.013134067189237,1.8277959527833965,-10.33855773598592